In [ ]:
import tkinter as tk
from tkinter import messagebox
from collections import deque

# ----- Water Jug Problem Implementation -----
def water_jug_solver(jug1, jug2, target):
    visited = set()
    queue = deque()
    actions = {}

    queue.append((0, 0, []))

    while queue:
        a, b, steps = queue.popleft()

        if a == target or b == target:
            steps.append(((a, b), "Goal Reached"))
            return steps

        if (a, b) in visited:
            continue

        visited.add((a, b))
        next_states = []

        next_states.append(((jug1, b), "Fill Jug 1"))
        next_states.append(((a, jug2), "Fill Jug 2"))
        next_states.append(((0, b), "Empty Jug 1"))
        next_states.append(((a, 0), "Empty Jug 2"))

        transfer = min(a, jug2 - b)
        next_states.append(((a - transfer, b + transfer), "Pour Jug 1 -> Jug 2"))
        transfer = min(b, jug1 - a)
        next_states.append(((a + transfer, b - transfer), "Pour Jug 2 -> Jug 1"))

        for (state, action) in next_states:
            if state not in visited:
                queue.append((state[0], state[1], steps + [((a, b), action)]))

    return None

# ----- GUI with Click-to-Advance Animation -----
class WaterJugGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Water Jug Problem")
        self.jug1_capacity = 0
        self.jug2_capacity = 0
        self.steps = []
        self.canvas = None
        self.step_index = 0
        self.progress = None
        self.create_input_screen()

    def create_input_screen(self):
        self.clear_window()
        tk.Label(self.root, text="Enter Jug Capacities and Target:").pack(pady=5)

        tk.Label(self.root, text="Jug 1 Capacity:").pack()
        self.jug1_entry = tk.Entry(self.root)
        self.jug1_entry.pack()

        tk.Label(self.root, text="Jug 2 Capacity:").pack()
        self.jug2_entry = tk.Entry(self.root)
        self.jug2_entry.pack()

        tk.Label(self.root, text="Target Volume:").pack()
        self.target_entry = tk.Entry(self.root)
        self.target_entry.pack()

        tk.Button(self.root, text="Solve", command=self.prepare_animation).pack(pady=10)

    def prepare_animation(self):
        try:
            jug1 = int(self.jug1_entry.get())
            jug2 = int(self.jug2_entry.get())
            target = int(self.target_entry.get())

            self.jug1_capacity = jug1
            self.jug2_capacity = jug2

            self.steps = water_jug_solver(jug1, jug2, target)
            self.clear_window()

            if not self.steps:
                tk.Label(self.root, text="No solution found.").pack()
                tk.Button(self.root, text="Try Another", command=self.create_input_screen).pack(pady=10)
                return

            self.canvas = tk.Canvas(self.root, width=400, height=300, bg="white")
            self.canvas.pack(pady=10)

            # Jug outlines
            self.jug1_rect = self.canvas.create_rectangle(100, 50, 150, 250, outline="black", width=2)
            self.jug2_rect = self.canvas.create_rectangle(250, 50, 300, 250, outline="black", width=2)
            # Jug fills
            self.jug1_fill = self.canvas.create_rectangle(100, 250, 150, 250, fill="#4DA6FF")
            self.jug2_fill = self.canvas.create_rectangle(250, 250, 300, 250, fill="#4DA6FF")

            # Labels
            self.canvas.create_text(125, 260, text="Jug 1")
            self.canvas.create_text(275, 260, text="Jug 2")

            # Step text and progress
            self.step_label = tk.Label(self.root, text="")
            self.step_label.pack()
            self.action_label = tk.Label(self.root, text="")
            self.action_label.pack()
            self.progress = tk.Label(self.root, text="")
            self.progress.pack()

            self.next_button = tk.Button(self.root, text="Next Step", command=self.next_step)
            self.next_button.pack(pady=10)

            self.step_index = 0
            self.update_visuals()

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter valid integers.")

    def next_step(self):
        if self.step_index < len(self.steps):
            self.update_visuals()
            self.step_index += 1
        else:
            self.step_label.config(text="Done!")
            self.action_label.config(text="")
            self.progress.config(text="100% complete")
            self.next_button.config(state="disabled")
            tk.Button(self.root, text="Try Another", command=self.create_input_screen).pack(pady=10)

    def update_visuals(self):
        (a, b), action = self.steps[self.step_index]

        self.step_label.config(text=f"Step {self.step_index + 1}: Jug1 = {a}, Jug2 = {b}")
        self.action_label.config(text=f"Action: {action}")
        self.progress.config(text=f"Progress: {int((self.step_index + 1)/len(self.steps)*100)}%")

        jug1_height = 200 * (a / self.jug1_capacity) if self.jug1_capacity else 0
        jug2_height = 200 * (b / self.jug2_capacity) if self.jug2_capacity else 0

        self.canvas.coords(self.jug1_fill, 100, 250 - jug1_height, 150, 250)
        self.canvas.coords(self.jug2_fill, 250, 250 - jug2_height, 300, 250)

    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()

if __name__ == '__main__':
    root = tk.Tk()
    app = WaterJugGUI(root)
    root.mainloop()